In [182]:
from os import listdir # list all files in a directory
PATH = "C:/Users/yishai and shira/bible/"
# listdir(PATH)

In [215]:
import pandas as pd # data manipulation
KING_JAMES = 't_kjv.csv' # a constant, the tail end of the file with the verses of the KJV bible
ds = pd.read_csv(PATH + KING_JAMES)[:10000] # load data into dataframe
# ds.tail() # print a few verses

In [216]:
KEY = 'key_english.csv' # a constant, the file with book numbers
df_key = pd.read_csv(PATH + KEY) # load data into dataframe 
book_nums = df_key[['b','n']] # select the columns you want
# nums.columns = ['Number','Book Name'] # rename the columns to be descriptive
# book_nums.head(5) # first few books. Note the first 39 are from Hebrew Bible

In [217]:
# book_nums['n'][0] # select the string of a book name

In [218]:
print('Where do you want to search? Type number, then press enter...\n')

book_num = int(input('Book: '))
chap = int(input("Chapter: "))
verse = int(input("Verse: "))

Where do you want to search? Type number, then press enter...

Book: 2
Chapter: 15
Verse: 1


In [219]:
def book_num_to_name(n):
    """
    given the index, produce the book name
    e.g. 1 results in genesis
    """
    return book_nums.loc[ book_nums['b'] == n ]['n'].iloc[0]
# book_num_to_name(2)

In [256]:
def book_id_to_name(verse_id):
    book_num = ds.loc[ds['id'] == verse_id            
                     ]['b'].iloc[0]
    return book_num_to_name(book_num)
# book_id_to_name(1002001)

In [221]:
book_choice = book_num_to_name(book_num)

In [222]:
print("Wait! Please, double check before proceeding.")
print("Your current choice is\n*** {} {}:{} ***".format(book_choice,chap,verse)) # Tell the user which bok he chose

Wait! Please, double check before proceeding.
Your current choice is
*** Exodus 15:1 ***


In [223]:
my_id = book_num*1000000 + chap*1000 + verse
# my_id

In [224]:
# my_id = 5012021 # choose sample index for verse
my_row = ds.loc[ ds['id'] == my_id ] # select the row of ds in 'id' matches my_id
# my_row

In [225]:
my_verse = my_row['t'].iloc[0] #
# print(type(my_verse))
print(my_verse)

Then sang Moses and the children of Israel this song unto the LORD, and spake, saying, I will sing unto the LORD, for he hath triumphed gloriously: the horse and his rider hath he thrown into the sea.


In [226]:
# TF IDF stands for "term frequency–inverse document frequency"
# it is a a numerical statistic that is intended to reflect how important a word is
# to a document in a collection or corpus.
from sklearn.feature_extraction.text import TfidfVectorizer # Convert a collection of raw documents to a matrix of TF-IDF features
from sklearn.metrics.pairwise import linear_kernel

In [227]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english') # create instance of tf odf vectorizer

In [228]:
tfidf_matrix = tf.fit_transform(ds['t']) # Learn vocabulary and idf, return term-document matrix.

In [229]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) # dot product

In [230]:
results = {} # dict where each key is an
# id in the list of verses
# and the entry for that key is
# a list id's for verses that are simliar the the verse of the key

for idx, row in ds.iterrows(): # 
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1] # numpy.ndarray
    #print(type(similar_indices))
    similar_rows = [(cosine_similarities[idx][i], ds['id'][i]) for i in similar_indices] # list of 
    #print(type(similar_rows))
    # First row is the row itself, so remove it.
    # Each dictionary entry is like: [(1,2), (3,4)], with each tuple being (score, row_id)
    results[row['id']] = similar_rows[1:]
    
print('done!')

done!


In [231]:
def get_verse_text(id):
    """
    get the words of the verse, given the id
    """
    return ds.loc[ds['id'] == id]['t'].values[0]
# get_verse_text(my_id)

In [232]:
def get_verse_num(verse_id):
    return ds.loc[ds['id'] == verse_id]['v'].iloc[0]
# get_verse_num(1001011)

In [233]:
def get_chap(verse_id):
    return ds.loc[ds['id'] == verse_id]['c'].iloc[0]
# get_chap(1002001)

In [234]:
def cit_to_id(book_num,chap,verse):
    """
    given a book number, chap number, and verse number, produce the verse id
    """ 
    return book_num*1000000 + chap*1000 + verse
# cit_to_id(1,1,1)

In [235]:
# reads the results out of the dictionary
def recommend(verse_id, num):
    my_info = 0
    print("The top {} similar verses to {} {}:{}\n\n{}\n".format(num, book_id_to_name(verse_id), get_chap(verse_id), get_verse_num(verse_id), get_verse_text(verse_id)))
    print("-------")
    recs = results[verse_id][:num] # the top num items listed in the recomendations for this id
    result = []
    for rec in recs:        
        rank = ''+str(recs.index(rec)+1)+'.)'
        citation = rec[1]
        book = book_id_to_name(citation)
        chap = get_chap(citation)
        verse = get_verse_num(citation)
        score = str(int(rec[0]*100))[:2]+ "%"
        text = get_verse_text(rec[1])
        print()
        print(rank,score,book,str(chap)+':'+str(verse))
        print()
        print(text)
user_input = cit_to_id(book_num,chap,verse)
recommend(verse_id=user_input, num=3)

The top 3 similar verses to Exodus 15:1

Then sang Moses and the children of Israel this song unto the LORD, and spake, saying, I will sing unto the LORD, for he hath triumphed gloriously: the horse and his rider hath he thrown into the sea.

-------

1.) 57% Exodus 15:21

And Miriam answered them, Sing ye to the LORD, for he hath triumphed gloriously; the horse and his rider hath he thrown into the sea.

2.) 12% Judges 5:3

Hear, O ye kings; give ear, O ye princes; I, even I, will sing unto the LORD; I will sing praise to the LORD God of Israel.

3.) 11% Exodus 39:42

According to all that the LORD commanded Moses, so the children of Israel made all the work.


In [252]:
type(ds['t'].iloc[0])

str

In [254]:
ds['t'].iloc[0] == 'In the beginning God created the heaven and the earth.'

True

In [264]:
ds['t'][0]

'In the beginning God created the heaven and the earth.'

In [275]:
df1 = ds.loc[ds['t']=='In the beginning God created the heaven and the earth.']
df1.head()

,id,b,c,v,t
0,1001001,1,1,1,In the beginning God created the heaven and th...


In [ ]:
ds.t.str.startswith('In')

In [302]:
# ds[ds.t.str.startswith('In')]

In [300]:
keyword = input("Type the word you'd like to find. Then press enter. Your choice: ")

Type the word you'd like to find. Then press enter. Your choice: song


In [303]:
keyword

'song'

In [304]:
df_search = ds[ds['t'].str.contains(keyword)]
# ds[ds['t'].str.contains("song")]

In [305]:
df_search.head()

,id,b,c,v,t
900,1031027,1,31,27,"Wherefore didst thou flee away secretly, and s..."
1921,2015001,2,15,1,Then sang Moses and the children of Israel thi...
1922,2015002,2,15,2,"The LORD is my strength and song, and he is be..."
4357,4021017,4,21,17,"Then Israel sang this song, Spring up, O well;..."
5747,5031019,5,31,19,"Now therefore write ye this song for you, and ..."


In [306]:
df_search[:5]

,id,b,c,v,t
900,1031027,1,31,27,"Wherefore didst thou flee away secretly, and s..."
1921,2015001,2,15,1,Then sang Moses and the children of Israel thi...
1922,2015002,2,15,2,"The LORD is my strength and song, and he is be..."
4357,4021017,4,21,17,"Then Israel sang this song, Spring up, O well;..."
5747,5031019,5,31,19,"Now therefore write ye this song for you, and ..."


In [ ]:
book = book_id_to_name(citation)
chap = get_chap(citation)
verse = get_verse_num(citation)


In [315]:
print("These verses contain your keyword '{}'.".format(keyword))
print()
for i in range(0,5):
    print(str(i+1)+'.',df_search.iloc[i]['t'])
    print()

These verses contain your keyword 'song'.

1. Wherefore didst thou flee away secretly, and steal away from me; and didst not tell me, that I might have sent thee away with mirth, and with songs, with tabret, and with harp?

2. Then sang Moses and the children of Israel this song unto the LORD, and spake, saying, I will sing unto the LORD, for he hath triumphed gloriously: the horse and his rider hath he thrown into the sea.

3. The LORD is my strength and song, and he is become my salvation: he is my God, and I will prepare him an habitation; my father's God, and I will exalt him.

4. Then Israel sang this song, Spring up, O well; sing ye unto it:

5. Now therefore write ye this song for you, and teach it the children of Israel: put it in their mouths, that this song may be a witness for me against the children of Israel.

